In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import xlrd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = "/content/copper_new_dataset_.csv"

# Read the Excel file
df = pd.read_csv(file_path)

In [4]:
df.head(2)

,id,item_date,quantity tons,customer,country,status,item type,application,thickness,width,material_ref,product_ref,delivery date,selling_price
0,EC06F063-9DF0-440C-8764-0B0C05A4F6AE,2021-04-01,54.151139,30156308.0,28.0,Won,W,10.0,2.0,1500.0,DEQ1 S460MC,1670798778,2021-07-01,854.0
1,4E5F4B3D-DDDF-499D-AFDE-A3227EC49425,2021-04-01,768.024839,30202938.0,25.0,Won,W,41.0,0.8,1210.0,104991,1668701718,2021-04-01,1047.0


In [5]:
df.shape

(181631, 14)

In [15]:
df = df[df['status'].isin(['Won', 'Lost'])]
len(df)

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer

Y = df['status']
X= df[['item type','application','width','country','customer','product_ref']]

# encoding categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X[['item type']])
X_ohe = ohe.fit_transform(X[['item type']]).toarray()

be = LabelBinarizer()
be.fit(Y)
y = be.fit_transform(Y)

# independent features after encoding
X = np.concatenate((X[['application','width','country','customer','product_ref']].values, X_ohe), axis=1)


# Normalizing the encoded data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
test_dataframe2 = pd.DataFrame(X)
test_dataframe2.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.861401,0.759539,-0.710173,-0.725072,1.667357,-0.013398,-0.055563,-0.18603,-0.731707,-0.002578,0.797518,-0.057979
1,0.895984,-0.330023,-0.832681,-0.283543,1.664434,-0.013398,-0.055563,-0.18603,-0.731707,-0.002578,0.797518,-0.057979
2,0.159016,-1.299357,-0.628500,-0.747276,-0.661157,-0.013398,-0.055563,-0.18603,-0.731707,-0.002578,-1.253891,17.247671
3,1.916401,0.071988,-0.546828,1.104922,1.664434,-0.013398,-0.055563,-0.18603,1.366667,-0.002578,-1.253891,-0.057979
4,-0.861401,2.638093,-0.710173,-0.201903,-0.661140,-0.013398,-0.055563,-0.18603,-0.731707,-0.002578,0.797518,-0.057979


In [17]:
test_dataframe3 = pd.DataFrame(y)
test_dataframe3.head()

,0
0,1
1,1
2,1
3,1
4,1


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
# test and train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


# Decision Tree Classifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)


# evalution metrics
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:", cm)

Accuracy: 0.8813773390500881
Confusion Matrix: [[ 4613  2182]
 [ 1387 21905]]


In [19]:
# Saving the model
import pickle
with open('cmodel.pkl', 'wb') as file:
    pickle.dump(dtc, file)
with open('cscaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('ct.pkl', 'wb') as f:
    pickle.dump(ohe, f)